### Importing dependencies

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow
import torchvision.models as models
import torch.nn.functional as F

import time
import os
import copy

plt.ion()   # interactive mode

https://pytorch.org/tutorials/beginner/transfer_learning_tutorial.html

We will use torchvision and torch.utils.data packages for loading the data.


In [2]:
batch_size = 128
img_size = 224
epochs = 3

In [3]:
traindir ='/mnt/ml-team/homes/rafal.jakubanis/blogpost2-resnet50/data_small/train'
valdir = '/mnt/ml-team/homes/rafal.jakubanis/blogpost2-resnet50/data_small/validation'


In [4]:
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])

In [5]:
# Data augmentation and normalization for training
# Just normalization for validation
data_transforms = {
    'train':
    transforms.Compose([
            transforms.Resize((img_size,img_size)),
            transforms.RandomResizedCrop(img_size,scale=(0.8,1),ratio=(1,1)),
            transforms.RandomHorizontalFlip(),
            transforms.RandomAffine(10, shear=(0,10)),
            transforms.ToTensor(),
            normalize
    ]),
    'validation':
    transforms.Compose([
        transforms.Resize((img_size,img_size)),
        transforms.ToTensor(),
        normalize
    ]),
}

data_dir = 'data_small'
image_datasets = {
    x: datasets.ImageFolder(os.path.join(data_dir, x), data_transforms[x])
    for x in ['train', 'validation']
}

dataloaders = {
    x: torch.utils.data.DataLoader(
        image_datasets[x], batch_size=256, shuffle=True, num_workers=4)
    for x in ['train', 'validation']
}
dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'validation']}

class_names = image_datasets['train'].classes

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [6]:
model= models.resnet50(pretrained=True).to(device)

In [7]:
# Don't update non-classifier learned features in the pretrained networks
for param in model.parameters():
    param.requires_grad = False

In [8]:
# # Let's replace the last fully-connected layer with a tiny sequential model
# # Note that newly constructed modules are trainable (requires_grad=True) by default.

model.fc = nn.Sequential(
            nn.Linear(2048, 256),
            nn.ReLU(inplace=True),
            nn.Linear(256, 2),
        ).to(device)

# for layer in model.parameters():
#     print(layer.requires_grad)

In [9]:
def train_model(model, criterion, optimizer, num_epochs=3):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'validation']:
            if phase == 'train':
#                 scheduler.step()
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history only in train mode
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                # loss.item() gets the a scalar value held in the loss.

                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                phase, epoch_loss, epoch_acc))

            # deep copy the best model
            if phase == 'validation' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())
                

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model

In [10]:
criterion = nn.CrossEntropyLoss()
# criterion = nn.BCEWithLogitsLoss()

optimizer_conv = optim.RMSprop(model.fc.parameters())

In [11]:
%%time

model_conv = train_model(model, criterion, optimizer_conv,
                        num_epochs=epochs)

Epoch 0/2
----------
train Loss: 300.8327 Acc: 0.6165
validation Loss: 0.0840 Acc: 0.9740
Epoch 1/2
----------
train Loss: 0.1965 Acc: 0.9450
validation Loss: 0.1062 Acc: 0.9750
Epoch 2/2
----------
train Loss: 0.1008 Acc: 0.9645
validation Loss: 0.0662 Acc: 0.9790
Training complete in 0m 49s
Best val Acc: 0.979000
CPU times: user 18.8 s, sys: 8.41 s, total: 27.2 s
Wall time: 49.1 s


### Save the model

In [12]:
model_conv.state_dict('/tmp')

AttributeError: 'str' object has no attribute '_metadata'

In [ ]:
model_conv.save_state_dict('fine_tuned_best_model.pt')

In [ ]:
state_dict = torch.load("output/best_model.pth.tar")

In [ ]:
eeeeeeee